In [217]:
import pandas as pd
import re
import numpy as np
import os
from fuzzywuzzy import fuzz
import ast
from nltk import ngrams
import math

In [206]:
df_food_posts = pd.read_csv(r'food_posts_preprocessed_new.csv')

In [207]:
len(df_food_posts)

561693

In [208]:
df_food_posts.head()

,Unnamed: 0,datetime,title,preprocessed_title,preprocessed_title_no_stopwords,preprocessed_title_final
0,0,2018-03-01 00:01:28,[I ate] Thinly-sliced Japanese Wagyu,thinly sliced japanese wagyu,thinly sliced japanese wagyu,thinly sliced japanese wagyu
1,1,2018-03-01 00:13:38,"I smoked a brisket and we made doughnuts, so t...",i smoked a brisket and we made doughnuts so t...,smoked brisket doughnuts happened,smoked brisket doughnuts happened
2,2,2018-03-01 00:14:56,[I Ate] Matcha Mochi Melonpan,matcha mochi melonpan,matcha mochi melonpan,matcha mochi melonpan
3,3,2018-03-01 00:15:10,[I made] this Wakanda/Black Panther inspired U...,this wakanda black panther inspired unicorn cake,wakanda black panther inspired unicorn cake,wakanda black panther inspired unicorn cake
4,4,2018-03-01 00:16:01,Homemade Flax seed dough. Caper [homemade],homemade flax seed dough caper,homemade flax seed dough caper,homemade flax seed dough caper


In [210]:
df_food_posts_1 = df_food_posts.drop(columns=['Unnamed: 0', 'preprocessed_title_no_stopwords', 'preprocessed_title_final'])

In [212]:
from gensim.parsing.preprocessing import remove_stopwords

In [215]:
df_food_posts_1['preprocessed_title'] = df_food_posts_1['preprocessed_title'].apply(lambda x: remove_stopwords(x) if type(x) is str else x)

In [211]:
df_food_posts_1.head()

,datetime,title,preprocessed_title
0,2018-03-01 00:01:28,[I ate] Thinly-sliced Japanese Wagyu,thinly sliced japanese wagyu
1,2018-03-01 00:13:38,"I smoked a brisket and we made doughnuts, so t...",i smoked a brisket and we made doughnuts so t...
2,2018-03-01 00:14:56,[I Ate] Matcha Mochi Melonpan,matcha mochi melonpan
3,2018-03-01 00:15:10,[I made] this Wakanda/Black Panther inspired U...,this wakanda black panther inspired unicorn cake
4,2018-03-01 00:16:01,Homemade Flax seed dough. Caper [homemade],homemade flax seed dough caper


In [216]:
df_food_posts_1.head()

,datetime,title,preprocessed_title
0,2018-03-01 00:01:28,[I ate] Thinly-sliced Japanese Wagyu,thinly sliced japanese wagyu
1,2018-03-01 00:13:38,"I smoked a brisket and we made doughnuts, so t...",smoked brisket doughnuts happened
2,2018-03-01 00:14:56,[I Ate] Matcha Mochi Melonpan,matcha mochi melonpan
3,2018-03-01 00:15:10,[I made] this Wakanda/Black Panther inspired U...,wakanda black panther inspired unicorn cake
4,2018-03-01 00:16:01,Homemade Flax seed dough. Caper [homemade],homemade flax seed dough caper


In [204]:
df_food_posts.author.value_counts().values.mean()

2.8124226083483124

In [200]:
df_food_posts.title.str.len().mean()

50.02958788931083

In [219]:
len(df_nutrition)

7083

In [293]:
df_nutrition = pd.read_json(r"..\..\FoodData_Central_sr_legacy_food_json_2021-10-28\FoodData_Central_sr_legacy_food_json_2021-10-28.json")

In [294]:
len(df_nutrition)

7793

In [175]:
df_nutrition.head()

,SurveyFoods
0,"{'foodClass': 'Survey', 'description': 'Milk, ..."
1,"{'foodClass': 'Survey', 'description': 'Milk, ..."
2,"{'foodClass': 'Survey', 'description': 'Milk, ..."
3,"{'foodClass': 'Survey', 'description': 'Milk, ..."
4,"{'foodClass': 'Survey', 'description': 'Milk, ..."


In [253]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [295]:
df_resampled = pd.DataFrame(columns = ['description', 'food_class', 'food_category', 'serving_size', 
                               'serving_size_unit', 'calories_100g', 'calories_unit', 'protein_100g',
                               'protein_unit', 'fat_100g', 'fat_unit','sugar_100g', 'sugar_unit', 'sodium_100g', 
                               'sodium_unit', 'cholesterol_100g', 'cholesterol_unit',
                               'carbohydrates_100g', 'carbohydrates_unit','fiber_100g',
                               'fiber_unit', 'protein_calories', 'fat_calories','carb_calories'])

In [296]:
for index, row in df_nutrition.iterrows():
    # Default values for converting grams to calories for protein, fat and carbohydrates 
    # (based on the official USDA information)
    protein_conversion_factor = 4.0
    fat_conversion_factor = 9.0
    carb_conversion_factor = 4.0
    
    description = row.values[0]['description']
    food_class = row.values[0]['foodClass']
    
    if 'servingSize' in row.values[0]:
        serving_size = row.values[0]['servingSize']
    elif 'inputFoods' in row.values[0] and 'amount' in row.values[0]['inputFoods']:
        serving_size = row.values[0]['inputFoods'][0]['amount']
    else:
        serving_size = 100
    
    if 'servingSizeUnit' in row.values[0]:
        unit = row.values[0]['servingSizeUnit']
    elif 'inputFoods' in row.values[0] and 'unit' in row.values[0]['inputFoods']:
        unit = row.values[0]['inputFoods'][0]['unit']
    else:
        unit = 'g'
    
    calories, protein, fat, sugar, sodium, fiber, cholesterol, carbohydrates, starch = -1, -1, -1, -1, -1, -1, -1, -1, -1
    
    calories_100, protein_100, fat_100, sugar_100, sodium_100, fiber_100, cholesterol_100, carbohydrates_100, starch_100 = -1, -1, -1, -1, -1, -1, -1, -1, -1
    
    calories_unit, protein_unit, fat_unit, sugar_unit, sodium_unit, fiber_unit, cholesterol_unit, carbohydrates_unit, starch_unit = '', '', '', '', '', '', '', '', ''
    
    protein_calories, fat_calories, carb_calories = 0, 0, 0
    food_category = ''
    
    if 'foodNutrients' in row.values[0]:
        n_list = row.values[0]['foodNutrients']
        for n in n_list:
            if n['nutrient']['name'].lower() == 'protein':
                protein_100 = n['amount']
                protein_unit = n['nutrient']['unitName']
                
            if 'fat'in n['nutrient']['name'].lower():
                fat_100 = n['amount']
                fat_unit = n['nutrient']['unitName']
                
            if 'carbohydrate' in n['nutrient']['name'].lower():
                carbohydrates_100 = n['amount']
                carbohydrates_unit = n['nutrient']['unitName']
                
            if  'energy' in n['nutrient']['name'].lower():
                calories_100 = n['amount']
                calories_unit = n['nutrient']['unitName']
                
            if 'sugar' in n['nutrient']['name'].lower():
                sugar_100 = n['amount']
                sugar_unit = n['nutrient']['unitName']
                
            if 'fiber' in n['nutrient']['name'].lower():
                fiber_100 = n['amount']
                fiber_unit = n['nutrient']['unitName']
                
            if 'sodium' in n['nutrient']['name'].lower():
                sodium_100 = n['amount']
                sodium_unit = n['nutrient']['unitName']
                
            if 'cholesterol' in n['nutrient']['name'].lower():
                cholesterol_100 = n['amount']
                cholesterol_unit = n['nutrient']['unitName']
            
            if 'starch' in n['nutrient']['name'].lower():
                starch_100 = n['amount']
                starch_unit = n['nutrient']['unitName']
                
    if 'labelNutrients' in row.values[0]:
        r = row.values[0]['labelNutrients']
        
        if 'calories' in r:
            calories = r['calories']['value']
        if 'protein' in r:
            protein = r['protein']['value']
        if 'fat' in r:
            fat = r['fat']['value']
        if 'sugars' in r:
            sugar = r['sugars']['value']
        if 'sodium' in r:
            sodium = r['sodium']['value']
        if 'cholesterol' in r:
            cholesterol = r['cholesterol']['value']
        if 'carbohydrates' in r:
            carbohydrates = r['carbohydrates']['value']
        if 'fiber' in r:
            fiber = r['fiber']['value']
        if 'starch' in r:
            starch = r['starch']['value']
        
    if 'nutrientConversionFactors' in row.values[0]:
        if len(row.values[0]['nutrientConversionFactors']) > 0:
            calorie_conversion_list = list(filter(lambda x: x['type'] == '.CalorieConversionFactor', row.values[0]['nutrientConversionFactors']))
            if len(calorie_conversion_list) > 0:
                if 'proteinValue' in calorie_conversion_list[0]:
                    protein_conversion_factor = calorie_conversion_list[0]['proteinValue']
                
                if 'fatValue' in calorie_conversion_list[0]:
                    fat_conversion_factor = calorie_conversion_list[0]['fatValue']
                
                if 'carbohydrateValue' in calorie_conversion_list[0]:
                    carb_conversion_factor = calorie_conversion_list[0]['carbohydrateValue']
    
    protein_calories = protein_100*protein_conversion_factor
    fat_calories = fat_100*fat_conversion_factor
    carb_calories = carbohydrates_100*carb_conversion_factor
        
    if 'foodCategory' in row.values[0]:
        food_category = row.values[0]['foodCategory']['description']
    if 'wweiaFoodCategory' in row.values[0]: 
        food_category = row.values[0]['wweiaFoodCategory']['wweiaFoodCategoryDescription']
        
    if calories_100 == 0 or calories_100 == -1:
        calories_100 = 0
        if fiber_100 != -1:
             calories_100 += fiber_100 
        if starch_100 != -1:
             calories_100 += starch_100 
        if sugar_100 != -1:
             calories_100 += sugar_100 
    
    df_new_row = pd.DataFrame({'description': [description], 'food_class': [food_class], 'food_category': [food_category], 
                               'serving_size': [serving_size], 
                               'serving_size_unit': [unit], 'calories_100g': [calories_100], 
                               'calories_unit': [calories_unit], 'protein_100g': [protein_100],
                               'protein_unit': [protein_unit], 'fat_100g': [fat_100], 'fat_unit': [fat_unit],
                               'sugar_100g': [sugar_100], 'sugar_unit': [sugar_unit], 'sodium_100g': [sodium_100], 
                               'sodium_unit': [sodium_unit], 'cholesterol_100g': [cholesterol_100], 
                               'cholesterol_unit': [cholesterol_unit], 
                               'carbohydrates_100g': [carbohydrates_100],
                                'carbohydrates_unit': [carbohydrates_unit],'fiber_100g':[fiber_100], 
                               'fiber_unit': [fiber_unit], 'protein_calories': [protein_calories], 'fat_calories': [fat_calories], 
                               'carb_calories': [carb_calories]})
    
    df_resampled = pd.concat([df_resampled, df_new_row])  


In [297]:
df_resampled.to_csv('sr_legacy_food_preprocessed.csv')

In [267]:
df_resampled = df_resampled.reset_index()

In [268]:
df_resampled = df_resampled.drop('index', axis=1)

In [270]:
df_resampled.head(3)

,description,class,category,serving,s_u,cals,cal_u,protein,p_u,fat,f_unit,sugar,su_u,sodium,so_u,cholesterol,ch_u,carbs,ca_u,fiber,fi_u,p_cal,f_cal,c_cal,f_u
0,"Milk, human",Survey,Human milk,100,g,70.0,kcal,1.03,g,0.497,NaN,6.89,g,17.0,mg,14.0,mg,6.89,g,0.0,g,4.12,4.473,27.56,g
1,"Milk, NFS",Survey,"Milk, reduced fat",100,g,51.0,kcal,3.34,g,0.065,NaN,4.89,g,39.0,mg,8.0,mg,4.87,g,0.0,g,13.36,0.585,19.48,g
2,"Milk, whole",Survey,"Milk, whole",100,g,60.0,kcal,3.28,g,0.108,NaN,4.81,g,38.0,mg,12.0,mg,4.67,g,0.0,g,13.12,0.972,18.68,g


In [180]:
df_resampled.to_csv('foundation_food_preprocessed.csv')

In [339]:
df_to_convert = pd.read_csv('foundation_food_preprocessed.csv')

In [340]:
def change_values_to_kcal(x):
    if isinstance(x['calories_unit'], str) and x['calories_unit'].lower() == 'kj':
        if isinstance(x['calories_100g'], float) and math.isnan(x['calories_100g']) == False:
            c = x['calories_100g']/4.184
            x['calories_100g'] = round(c, 2)
        x['calories_unit'] = 'kcal'
        return x
    return x

In [341]:
df_converted = df_to_convert.apply(lambda x: change_values_to_kcal(x), axis=1)

In [332]:
df_converted.head()

,Unnamed: 0,description,food_class,food_category,serving_size,serving_size_unit,calories_100g,calories_unit,protein_100g,protein_unit,fat_100g,fat_unit,sugar_100g,sugar_unit,sodium_100g,sodium_unit,cholesterol_100g,cholesterol_unit,carbohydrates_100g,carbohydrates_unit,fiber_100g,fiber_unit,protein_calories,fat_calories,carb_calories
0,0,"Milk, human",Survey,Human milk,100,g,70.0,kcal,1.03,g,0.497,g,6.89,g,17.0,mg,14.0,mg,6.89,g,0.0,g,4.12,4.473,27.56
1,0,"Milk, NFS",Survey,"Milk, reduced fat",100,g,51.0,kcal,3.34,g,0.065,g,4.89,g,39.0,mg,8.0,mg,4.87,g,0.0,g,13.36,0.585,19.48
2,0,"Milk, whole",Survey,"Milk, whole",100,g,60.0,kcal,3.28,g,0.108,g,4.81,g,38.0,mg,12.0,mg,4.67,g,0.0,g,13.12,0.972,18.68
3,0,"Milk, low sodium, whole",Survey,"Milk, whole",100,g,61.0,kcal,3.10,g,0.128,g,4.46,g,3.0,mg,14.0,mg,4.46,g,0.0,g,12.40,1.152,17.84
4,0,"Milk, calcium fortified, whole",Survey,"Milk, whole",100,g,60.0,kcal,3.28,g,0.108,g,4.81,g,38.0,mg,12.0,mg,4.67,g,0.0,g,13.12,0.972,18.68


In [186]:
def edit_nutrition_for_serving_size(x):
    p = 1
    if x['serving_size'] != 100:
        if x['serving_size'] == 0:
            p = 100/0.0001
        else:
            p = 100/x['serving_size']
        if isinstance(x['calories_serving_size'], float) and math.isnan(x['calories_serving_size']) == False:
            x['calories_100g'] = round(x['calories_serving_size'] * p, 2)
        
        if isinstance(x['carbohydrates_serving_size'], float) and math.isnan(x['carbohydrates_serving_size']) == False:
            x['carbohydrates_100g'] = round(x['carbohydrates_serving_size'] * p, 2)
        
        if isinstance(x['cholesterol_serving_size'], float) and math.isnan(x['cholesterol_serving_size']) == False:
            x['cholesterol_100g'] = round(x['cholesterol_serving_size'] * p, 2)
        
        if isinstance(x['fat_serving_size'], float) and math.isnan(x['fat_serving_size']) == False:
            x['fat_100g'] = round(x['fat_serving_size'] * p, 2)
        
        if isinstance(x['fiber_serving_size'], float) and math.isnan(x['fiber_serving_size']) == False:
            x['fiber_100g'] = round(x['fiber_serving_size'] * p, 2)
        
        if isinstance(x['protein_serving_size'], float) and math.isnan(x['protein_serving_size']) == False:
            x['protein_100g'] = round(x['protein_serving_size'] * p, 2)
        
        if isinstance(x['sodium_serving_size'], float) and math.isnan(x['sodium_serving_size']) == False:
            x['sodium_100g'] = round(x['sodium_serving_size'] * p, 2)
        
        if isinstance(x['sugar_serving_size'], float) and math.isnan(x['sugar_serving_size']) == False:
            x['sugar_100g'] = round(x['sugar_serving_size'] * p, 2)
    return x
        

In [187]:
df_converted_1 = df_converted.apply(lambda x: edit_nutrition_for_serving_size(x), axis=1)

In [342]:
df_converted['description'] = df_converted['description'].apply(lambda x: re.sub(r'[^\w]', ' ', x))

In [343]:
df_converted['description'] = df_converted['description'].apply(lambda x: re.sub('_', ' ', x))

In [344]:
df_converted['description'] = df_converted['description'].apply(lambda x: re.sub(' +', ' ', x))

In [345]:
df_converted['description'] = df_converted['description'].apply(lambda x: x.lower())

In [346]:
df_converted.head()

,Unnamed: 0,description,food_class,food_category,serving_size,serving_size_unit,calories_100g,calories_unit,protein_100g,protein_unit,fat_100g,fat_unit,sugar_100g,sugar_unit,sodium_100g,sodium_unit,cholesterol_100g,cholesterol_unit,carbohydrates_100g,carbohydrates_unit,fiber_100g,fiber_unit,protein_calories,fat_calories,carb_calories
0,0,hummus commercial,FinalFood,Legumes and Legume Products,100,g,229.00,kcal,7.35,g,16.10,g,0.34,g,438.0,mg,-1.0,NaN,13.90,g,5.4,g,25.5045,134.7570,56.5730
1,0,tomatoes grape raw,FinalFood,Vegetables and Vegetable Products,100,g,27.01,kcal,0.83,g,0.63,g,-1.00,NaN,6.0,mg,-1.0,NaN,5.51,g,2.1,g,2.0252,5.2731,19.6707
2,0,beans snap green canned regular pack drained s...,FinalFood,Vegetables and Vegetable Products,100,g,20.55,kcal,1.04,g,0.39,g,1.29,g,282.0,mg,-1.0,NaN,4.11,g,-1.0,NaN,2.5376,3.2643,14.6727
3,0,frankfurter beef unheated,FinalFood,Sausages and Luncheon Meats,100,g,313.10,kcal,11.70,g,26.00,g,1.26,g,872.0,mg,-1.0,NaN,2.89,g,-1.0,NaN,49.9590,234.5200,11.1843
4,0,nuts almonds dry roasted with salt added,FinalFood,Nut and Seed Products,100,g,619.02,kcal,20.40,g,53.40,g,4.17,g,256.0,mg,-1.0,NaN,16.20,g,11.0,g,70.7880,446.9580,65.9340


In [347]:
df_converted.to_csv('foundation_food_preprocessed_1.csv')

In [348]:
df_1 = pd.read_csv('survey_food_preprocessed_1.csv')
df_2 = pd.read_csv('sr_legacy_food_preprocessed_1.csv')
df_3 = pd.read_csv('foundation_food_preprocessed_1.csv')

In [349]:
df_4 = pd.concat([df_1, df_2, df_3], axis = 0)
df_4.head()

,Unnamed: 0.1,Unnamed: 0,description,food_class,food_category,serving_size,serving_size_unit,calories_100g,calories_unit,protein_100g,protein_unit,fat_100g,fat_unit,sugar_100g,sugar_unit,sodium_100g,sodium_unit,cholesterol_100g,cholesterol_unit,carbohydrates_100g,carbohydrates_unit,fiber_100g,fiber_unit,protein_calories,fat_calories,carb_calories
0,0,0,milk human,Survey,Human milk,100,g,70.0,kcal,1.03,g,0.497,g,6.89,g,17.0,mg,14.0,mg,6.89,g,0.0,g,4.12,4.473,27.56
1,1,0,milk nfs,Survey,"Milk, reduced fat",100,g,51.0,kcal,3.34,g,0.065,g,4.89,g,39.0,mg,8.0,mg,4.87,g,0.0,g,13.36,0.585,19.48
2,2,0,milk whole,Survey,"Milk, whole",100,g,60.0,kcal,3.28,g,0.108,g,4.81,g,38.0,mg,12.0,mg,4.67,g,0.0,g,13.12,0.972,18.68
3,3,0,milk low sodium whole,Survey,"Milk, whole",100,g,61.0,kcal,3.10,g,0.128,g,4.46,g,3.0,mg,14.0,mg,4.46,g,0.0,g,12.40,1.152,17.84
4,4,0,milk calcium fortified whole,Survey,"Milk, whole",100,g,60.0,kcal,3.28,g,0.108,g,4.81,g,38.0,mg,12.0,mg,4.67,g,0.0,g,13.12,0.972,18.68


In [350]:
df_4.to_csv('nutritional_database_combined_28_05.csv')


In [403]:
already_results = pd.read_csv('upgraded_results_for_timeseries_analysis.csv')
nutritional_database = pd.read_csv('nutritional_database_combined_28_05.csv')

In [404]:
len(already_results)

561550

In [407]:
already_results['index_of_usda_in_json_list'] = already_results['index_of_usda_in_json'].apply(lambda x: ast.literal_eval(x))

In [408]:
already_results['closest_match_from_usda_list'] = already_results['closest_match_from_usda'].apply(lambda x: ast.literal_eval(x))

In [418]:
nutritional_database = nutritional_database.drop(columns = ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'])

In [415]:
already_results.keys()

Index(['food_post_title', 'food_post_date', 'closest_match_from_usda',
       'protein', 'fat', 'sugar', 'carbohydrates', 'sodium', 'cholesterol',
       'fiber', 'protein_cals', 'fat_cals', 'carb_cals',
       'index_of_usda_in_json', 'index_of_usda_in_json_list',
       'closest_match_from_usda_list', 'calories'],
      dtype='object')

In [419]:
nutritional_database.iloc[0]

description           milk human
food_class                Survey
food_category         Human milk
serving_size                 100
serving_size_unit              g
calories_100g               70.0
calories_unit               kcal
protein_100g                1.03
protein_unit                   g
fat_100g                   0.497
fat_unit                       g
sugar_100g                  6.89
sugar_unit                     g
sodium_100g                 17.0
sodium_unit                   mg
cholesterol_100g            14.0
cholesterol_unit              mg
carbohydrates_100g          6.89
carbohydrates_unit             g
fiber_100g                   0.0
fiber_unit                     g
protein_calories            4.12
fat_calories               4.473
carb_calories              27.56
Name: 0, dtype: object

In [416]:
already_results.iloc[3]

food_post_title                                single serve sugar baby watermelon
food_post_date                                                2019-09-29 16:02:31
closest_match_from_usda         ['Fast foods  cheeseburger  single  large patt...
protein                                                                  8.543636
fat                                                                      2.021864
sugar                                                                   10.293333
carbohydrates                                                           18.650227
sodium                                                                 280.454545
cholesterol                                                             22.590909
fiber                                                                    0.962791
protein_cals                                                            34.370636
fat_cals                                                                17.456117
carb_cals       

In [432]:
upgraded_results = pd.DataFrame(columns = ['food_post_title', 'food_post_date',
                                           'protein', 'fat', 'sugar', 'calories',
                                            'carbohydrates', 'sodium', 'cholesterol',
                                           'fiber', 'protein_cals', 'fat_cals',
                                           'carb_cals',
                                           'index_of_usda_in_json_list', 'closest_match_from_usda_list',
                                          'food_categories'])

In [433]:
for index, row in already_results.iterrows():
    if index%1000 == 0:
        print('index: ' + str(index))
    list_of_indices = row['index_of_usda_in_json_list']
    #print('list_of_indices: ' + str(list_of_indices))
    list_of_matches = row['closest_match_from_usda_list']
    #print('list_of_matches: ' + str(list_of_matches))
    list_of_food_categories = []
    for ind_n, ind_v in enumerate(list_of_indices):
        match = nutritional_database.iloc[ind_v]
        #print('match: ' + str(match))
        first_name = re.sub(' +', ' ', list_of_matches[ind_n].lower())
        if match['description'] == first_name:
            list_of_food_categories.append(match['food_category'])
            
        else:
            print('no match for: ' + str(match['description']) + ' ----and---- ' + str(list_of_matches[ind_n].lower()) + 'in index: ' + str(index))
            
   
    
    df_new_row = pd.DataFrame({'food_post_title': [row['food_post_title']], 'food_post_date': [row['food_post_date']], 
                               'closest_match_from_usda_list': [list_of_matches],
                                'calories': [row['calories']], 'protein': [row['protein']], 'fat': [row['fat']], 
                               'sugar': [row['sugar']],
                                'carbohydrates':[row['carbohydrates']], 'sodium': [row['sodium']], 
                               'cholesterol': [row['cholesterol']],
                                'fiber': [row['fiber']], 'protein_cals': [row['protein_cals']], 'fat_cals': [row['fat_cals']],
                                'carb_cals': [row['carb_cals']], 'index_of_usda_in_json_list': [list_of_indices], 
                               'food_categories': [list_of_food_categories]})
    
    upgraded_results = pd.concat([upgraded_results, df_new_row]) 
        

index: 0
index: 1000
index: 2000
index: 3000
index: 4000
index: 5000
index: 6000
index: 7000
index: 8000
index: 9000
index: 10000
index: 11000
index: 12000
index: 13000
index: 14000
index: 15000
index: 16000
index: 17000
index: 18000
index: 19000
index: 20000
index: 21000
index: 22000
index: 23000
index: 24000
index: 25000
index: 26000
index: 27000
index: 28000
index: 29000
index: 30000
index: 31000
index: 32000
index: 33000
index: 34000
index: 35000
index: 36000
index: 37000
index: 38000
index: 39000
index: 40000
index: 41000
index: 42000
index: 43000
index: 44000
index: 45000
index: 46000
index: 47000
index: 48000
index: 49000
index: 50000
index: 51000
index: 52000
index: 53000
index: 54000
index: 55000
index: 56000
index: 57000
index: 58000
index: 59000
index: 60000
index: 61000
index: 62000
index: 63000
index: 64000
index: 65000
index: 66000
index: 67000
index: 68000
index: 69000
index: 70000
index: 71000
index: 72000
index: 73000
index: 74000
index: 75000
index: 76000
index: 77000

In [434]:
upgraded_results.head()

,food_post_title,food_post_date,protein,fat,sugar,calories,carbohydrates,sodium,cholesterol,fiber,protein_cals,fat_cals,carb_cals,index_of_usda_in_json_list,closest_match_from_usda_list,food_categories
0,clean fridge frittata,2019-09-29 15:42:15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,[],[],[]
0,poached eggs bacon mushrooms toasted sourdough...,2019-09-29 16:01:41,19.196491,1.464982,1.004912,183.433158,7.41807,496.701754,82.385965,0.438596,76.864912,13.185933,29.668295,"[1263, 1270, 1288, 1305, 1315, 1317, 1336, 134...","[Barracuda steamed or poached, Carp steamed ...","[Fish, Fish, Fish, Fish, Fish, Fish, Fish, Fis..."
0,spicy miso tsukemen,2019-09-29 16:02:26,12.035,15.2125,0.505,198.145,1.295,512.5,41.5,0.2,51.056,133.7835,5.1722,"[2961, 13004]","[Soybean soup miso broth, Cheese goat semis...","[Soups, Dairy and Egg Products]"
0,single serve sugar baby watermelon,2019-09-29 16:02:31,8.543636,2.021864,10.293333,183.341136,18.650227,280.454545,22.590909,0.962791,34.370636,17.456117,74.363318,"[9865, 10261, 10262, 10263, 10264, 10265, 1028...",[Fast foods cheeseburger single large patty...,"[Fast Foods, Fast Foods, Fast Foods, Fast Food..."
0,chocolate chip cookie dough cheesecake,2019-09-29 16:03:06,7.39587,6.511587,26.244,386.065435,51.198478,464.23913,19.444444,1.823256,28.76287,57.23013,201.057391,"[3356, 3460, 3461, 3462, 3463, 3494, 3504, 354...","[Muffin chocolate chip, Cookie chocolate chi...","[Biscuits, muffins, quick breads, Cookies and ..."


In [392]:
len(upgraded_results)

561550

In [435]:
upgraded_results.to_csv('upgraded_results_for_timeseries_analysis_with_food_category.csv')

In [4]:
new_df = pd.read_csv('nutritional_database_combined_28_05.csv')

In [84]:
new_df.head()

,Unnamed: 0,description,serving_size,calories_100g,protein_100g,fat_100g,sugar_100g,sodium_100g,cholesterol_100g,carbohydrates_100g,fiber_100g
0,0,"Hummus, commercial",100.0,229.00,7.35,16.10,0.34,438.0,NaN,13.90,5.4
1,1,"Tomatoes, grape, raw",100.0,27.01,0.83,0.63,NaN,6.0,NaN,5.51,2.1
2,2,"Beans, snap, green, canned, regular pack, drai...",100.0,20.55,1.04,0.39,1.29,282.0,NaN,4.11,NaN
3,3,"Frankfurter, beef, unheated",100.0,313.10,11.70,26.00,1.26,872.0,NaN,2.89,NaN
4,4,"Nuts, almonds, dry roasted, with salt added",100.0,619.02,20.40,53.40,4.17,256.0,NaN,16.20,11.0


In [85]:
words = new_df['description']

In [86]:
len(words)

15035

In [61]:
def create_ngrams(sentence):
    new_list = []
    if type(sentence) is str:
        n = len(sentence)

        for i in range(n,0,-1):
            grams = ngrams(sentence.split(), i)
            for g in grams:
                new_list.append(' '.join(list(g)))
    return new_list

In [62]:
df_food_posts['ngrams_title'] = df_food_posts['preprocessed_title_nouns'].apply(lambda x: create_ngrams(x))

In [63]:
df_food_posts.head()

,Unnamed: 0,id,url,datetime,title,author,num_comments,thumbnail,domain,preprocessed_title,preprocessed_title_nouns,preprocessed_title_nouns_no_duplicates,preprocessed_title_nouns_final,ngrams_title
0,0,810ib4,https://i.redd.it/ynh08qx9b1j01.jpg,2018-03-01 00:01:28,[I ate] Thinly-sliced Japanese Wagyu,Mike11888,125,https://b.thumbs.redditmedia.com/lEroJcEue-KPY...,i.redd.it,ate thinlysliced japanese wagyu,wagyu,['wagyu'],[wagyu],[wagyu]
1,1,810ljv,https://i.redd.it/kobpdjsed1j01.jpg,2018-03-01 00:13:38,"I smoked a brisket and we made doughnuts, so t...",Chef6288,0,default,i.redd.it,smoked brisket doughnuts happened prochef,brisket doughnuts,"['brisket', 'doughnuts']","[brisket, doughnuts]","[brisket doughnuts, brisket, doughnuts]"
2,2,810lvl,https://i.redd.it/ddugaidod1j01.jpg,2018-03-01 00:14:56,[I Ate] Matcha Mochi Melonpan,danceyourdeath,0,https://b.thumbs.redditmedia.com/EA7TutwVLakIt...,i.redd.it,ate matcha mochi melonpan,matcha mochi melonpan,"['matcha', 'mochi', 'melonpan']","[matcha, mochi, melonpan]","[matcha mochi melonpan, matcha mochi, mochi me..."
3,3,810lx7,https://i.redd.it/yqax4gypd1j01.jpg,2018-03-01 00:15:10,[I made] this Wakanda/Black Panther inspired U...,[deleted],0,default,i.redd.it,wakandablack panther inspired unicorn cake,wakandablack panther cake,"['wakandablack', 'panther', 'cake']","[wakandablack, panther, cake]","[wakandablack panther cake, wakandablack panth..."
4,4,810m58,https://imgur.com/m9BDe1F,2018-03-01 00:16:01,Homemade Flax seed dough. Caper [homemade],PassionateSizzle,1,default,imgur.com,homemade flax seed dough caper homemade,flax seed caper,"['flax', 'seed', 'caper']","[flax, seed, caper]","[flax seed caper, flax seed, seed caper, flax,..."


In [12]:
df_food_posts_ngrams = pd.read_csv('food_posts_with_ngrams.csv')

In [64]:
#df_food_posts_ngrams.head()

In [14]:
df_food_posts_ngrams.iloc[0]['ngrams_title']

"['ate thinlysliced japanese wagyu', 'ate thinlysliced japanese', 'thinlysliced japanese wagyu', 'ate thinlysliced', 'thinlysliced japanese', 'japanese wagyu', 'ate', 'thinlysliced', 'japanese', 'wagyu']"

In [67]:
df_food_titles_similar = pd.DataFrame(columns = ['food_post_title', 'closest_match_from_usda'])

In [70]:
for index, row in df_food_posts[:20].iterrows():
    name = []
    row_parts = []
    for r in row['ngrams_title']:
        for i, w in enumerate(words):
            if re.search(r, w):
                name.append(w)
                row_parts.append(r)
    
    df_new_row = pd.DataFrame({'food_post_title': [row_parts], 'closest_match_from_usda': [name]})
    
    df_food_titles_similar = pd.concat([df_food_titles_similar, df_new_row])  

In [71]:
df_food_titles_similar.head()

,food_post_title,closest_match_from_usda
0,[],[]
0,"[brisket, brisket, brisket, brisket, brisket, ...","[Beef brisket, cooked, NS as to fat eaten, Bee..."
0,[],[]
0,"[brisket, brisket, brisket, brisket, brisket, ...","[Beef brisket, cooked, NS as to fat eaten, Bee..."
0,[],[]


In [95]:
df_food_titles_similar.to_csv('compare_title_match_with_usda_2.csv')

In [7]:
def remove_words(x):
    y = ''
    if type(x) is not float:
        y = ' '.join(x)
    return y

In [5]:
df_food_posts['preprocessed_title_nouns_final'] = df_food_posts['preprocessed_title_nouns_final'].apply(lambda x: ast.literal_eval(x))

In [8]:
df_food_posts['preprocessed_title_nouns'] = df_food_posts['preprocessed_title_nouns_final'].apply(lambda x: remove_words(x))

In [14]:
len(df_food_posts)

556716

In [143]:
df_food_posts.head()

,Unnamed: 0,id,url,datetime,title,author,num_comments,thumbnail,domain,preprocessed_title,preprocessed_title_nouns,preprocessed_title_nouns_no_duplicates,preprocessed_title_nouns_final,ngrams_title
0,0,810ib4,https://i.redd.it/ynh08qx9b1j01.jpg,2018-03-01 00:01:28,[I ate] Thinly-sliced Japanese Wagyu,Mike11888,125,https://b.thumbs.redditmedia.com/lEroJcEue-KPY...,i.redd.it,ate thinlysliced japanese wagyu,wagyu,['wagyu'],[wagyu],[wagyu]
1,1,810ljv,https://i.redd.it/kobpdjsed1j01.jpg,2018-03-01 00:13:38,"I smoked a brisket and we made doughnuts, so t...",Chef6288,0,default,i.redd.it,smoked brisket doughnuts happened prochef,brisket doughnuts,"['brisket', 'doughnuts']","[brisket, doughnuts]","[brisket doughnuts, brisket, doughnuts]"
2,2,810lvl,https://i.redd.it/ddugaidod1j01.jpg,2018-03-01 00:14:56,[I Ate] Matcha Mochi Melonpan,danceyourdeath,0,https://b.thumbs.redditmedia.com/EA7TutwVLakIt...,i.redd.it,ate matcha mochi melonpan,matcha mochi melonpan,"['matcha', 'mochi', 'melonpan']","[matcha, mochi, melonpan]","[matcha mochi melonpan, matcha mochi, mochi me..."
3,3,810lx7,https://i.redd.it/yqax4gypd1j01.jpg,2018-03-01 00:15:10,[I made] this Wakanda/Black Panther inspired U...,[deleted],0,default,i.redd.it,wakandablack panther inspired unicorn cake,wakandablack panther cake,"['wakandablack', 'panther', 'cake']","[wakandablack, panther, cake]","[wakandablack panther cake, wakandablack panth..."
4,4,810m58,https://imgur.com/m9BDe1F,2018-03-01 00:16:01,Homemade Flax seed dough. Caper [homemade],PassionateSizzle,1,default,imgur.com,homemade flax seed dough caper homemade,flax seed caper,"['flax', 'seed', 'caper']","[flax, seed, caper]","[flax seed caper, flax seed, seed caper, flax,..."


In [111]:
df_food_titles_similar = pd.DataFrame(columns = ['food_post_title', 'food_post_date', 'closest_match_from_usda', 'calories_100g', 'protein_100g', 'fat_100g', 'sugar_100g', 'carbohydrates_100g', 'sodium_100g', 'cholesterol_100g', 'fiber_100g' ])

In [112]:
for index, row in df_food_posts[:2000].iterrows():
    counter, calories_val, protein_val, fat_val, sugar_val, carb_val, sodium_val, choles_val, fiber_val = 0,0,0,0,0,0,0,0,0
    title_matches, ind_list = [], []
    tmp_title = row['preprocessed_title']
    
    while tmp_title != '':
        result = sorted([(fuzz.token_set_ratio(tmp_title,j),j) for j in list(new_df['description'])], key=lambda x: x[0] , reverse=True)
        maximal_value = result[0][0]
        if maximal_value == 0:
            break
        final_result = [i for i in result if i[0] == maximal_value]
        tmp_result_name = ''

        if len(final_result) > 1:
            for f in final_result:
                tmp_result_name += f[1]
                counter+=1
                print('counter: '+ str(counter))
                
                ind = list(new_df['description']).index(f[1])
                ind_list.append(ind)
                #print('ind: ' + str(ind))
                #print(new_df.iloc[ind])
                
                if not pd.isnull(new_df.iloc[ind]['calories_100g']):
                    calories_val+=new_df.iloc[ind]['calories_100g']
            
                if not pd.isnull(new_df.iloc[ind]['protein_100g']):
                    protein_val+=new_df.iloc[ind]['protein_100g']

                if not pd.isnull(new_df.iloc[ind]['fat_100g']):
                    fat_val+=new_df.iloc[ind]['fat_100g']

                if not pd.isnull(new_df.iloc[ind]['sugar_100g']):
                    sugar_val+=new_df.iloc[ind]['sugar_100g']

                if not pd.isnull(new_df.iloc[ind]['carbohydrates_100g']):
                    carb_val+=new_df.iloc[ind]['carbohydrates_100g']

                if not pd.isnull(new_df.iloc[ind]['sodium_100g']):
                    sodium_val+=new_df.iloc[ind]['sodium_100g'] 

                if not pd.isnull(new_df.iloc[ind]['cholesterol_100g']):
                    choles_val+=new_df.iloc[ind]['cholesterol_100g']

                if not pd.isnull(new_df.iloc[ind]['fiber_100g']):
                    fiber_val+=new_df.iloc[ind]['fiber_100g']
            
            title_matches.append(tmp_result_name)
            calories_val=round(calories_val/counter, 2)
            protein_val=round(protein_val/counter, 2)
            fat_val=round(fat_val/counter, 2)
            sugar_val=round(sugar_val/counter, 2)
            carb_val=round(carb_val/counter, 2)
            sodium_val=round(sodium_val/counter, 2)
            choles_val=round(choles_val/counter, 2)
            fiber_val=round(fiber_val/counter, 2)
            
            counter = 1
            
            # Find how much of the title matches
            t1 = set(tmp_title.lower().split(" "))
            t2 = set(re.split('; |, | |\n', f[1].lower()))
            matching_words = list(t1&t2)
            matching_words = [x for x in matching_words if x and x != ' ']
        
            if len(matching_words) == 0:
                tmp_title = ''
                
            for mw in matching_words:
                tmp_title = tmp_title.replace(mw, '')
        else:
            final_result = final_result[0]
            tmp_result_name = final_result[1]
            title_matches.append(tmp_result_name)
            ind = list(new_df['description']).index(tmp_result_name)
            ind_list.append(ind)
            print(ind)

            # Find how much of the title matches
            t1 = set(tmp_title.lower().split(" "))
            t2 = set(re.split('; |, | |\n', final_result[1].lower()))
            matching_words = list(t1&t2)
            matching_words = [x for x in matching_words if x and x != ' ']

            if len(matching_words) == 0:
                tmp_title = ''

            # Add nutritional values
            counter+=1
            if not pd.isnull(new_df.iloc[ind]['calories_100g']):
                calories_val+=new_df.iloc[ind]['calories_100g']
            
            if not pd.isnull(new_df.iloc[ind]['protein_100g']):
                protein_val+=new_df.iloc[ind]['protein_100g']
            
            if not pd.isnull(new_df.iloc[ind]['fat_100g']):
                fat_val+=new_df.iloc[ind]['fat_100g']
            
            if not pd.isnull(new_df.iloc[ind]['sugar_100g']):
                sugar_val+=new_df.iloc[ind]['sugar_100g']
            
            if not pd.isnull(new_df.iloc[ind]['carbohydrates_100g']):
                carb_val+=new_df.iloc[ind]['carbohydrates_100g']
            
            if not pd.isnull(new_df.iloc[ind]['sodium_100g']):
                sodium_val+=new_df.iloc[ind]['sodium_100g'] 
            
            if not pd.isnull(new_df.iloc[ind]['cholesterol_100g']):
                choles_val+=new_df.iloc[ind]['cholesterol_100g']
            
            if not pd.isnull(new_df.iloc[ind]['fiber_100g']):
                fiber_val+=new_df.iloc[ind]['fiber_100g']

            for mw in matching_words:
                tmp_title = tmp_title.replace(mw, '')

            # compute average calories
            if counter > 0:
                calories_val=round(calories_val/counter, 2)
                protein_val=round(protein_val/counter, 2)
                fat_val=round(fat_val/counter, 2)
                sugar_val=round(sugar_val/counter, 2)
                carb_val=round(carb_val/counter, 2)
                sodium_val=round(sodium_val/counter, 2)
                choles_val=round(choles_val/counter, 2)
                fiber_val=round(fiber_val/counter, 2)
    
    df_new_row = pd.DataFrame({'food_post_title': row['preprocessed_title'], 'food_post_date': row['datetime'], 'closest_match_from_usda':[title_matches],
                               'index_of_usda_in_json': [ind_list],
                               'calories_100g': [calories_val], 'protein_100g': [protein_val], 
                               'fat_100g': [fat_val], 'sugar_100g': [sugar_val], 'carbohydrates_100g': [carb_val], 
                               'sodium_100g': [sodium_val], 'cholesterol_100g': [choles_val], 
                               'fiber_100g': [fiber_val]})
    
    df_food_titles_similar = pd.concat([df_food_titles_similar, df_new_row])  

3694
1607
counter: 1
counter: 2
3856
4427
counter: 1
counter: 2
counter: 3
counter: 2
counter: 3
counter: 4
4390
3866
counter: 1
counter: 2
counter: 3
572
7730
10430
11559
counter: 3
counter: 4
counter: 5
counter: 6
13294
counter: 2
counter: 3
4390
2437
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
counter: 26
counter: 27
2861
4986
2832
5337
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
counter: 26
coun

counter: 43
counter: 44
counter: 45
counter: 46
counter: 47
counter: 48
counter: 49
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
1997
13671
5986
counter: 2
counter: 3
counter: 2
counter: 3
counter: 4
counter: 5
counter: 2
counter: 3
3137
12482
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 1
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
counter: 26
counter: 27
counter: 28
counter: 29
counter: 30
counter: 31
counter: 32
counter: 33
counter: 34
counter: 35
counter: 36

counter: 104
counter: 105
counter: 106
counter: 107
counter: 108
counter: 109
counter: 110
counter: 111
counter: 112
counter: 113
counter: 114
counter: 115
counter: 116
counter: 117
counter: 118
counter: 119
counter: 120
counter: 121
counter: 122
counter: 123
counter: 124
counter: 125
counter: 126
counter: 127
counter: 128
counter: 129
counter: 130
counter: 131
counter: 132
counter: 133
counter: 134
counter: 135
counter: 136
counter: 137
counter: 138
counter: 139
counter: 140
counter: 141
counter: 142
counter: 143
counter: 144
counter: 145
counter: 146
counter: 147
counter: 148
counter: 149
counter: 150
counter: 151
counter: 152
counter: 153
counter: 154
counter: 155
counter: 156
counter: 157
counter: 158
counter: 159
counter: 160
counter: 161
counter: 162
counter: 163
counter: 164
counter: 165
counter: 166
counter: 167
counter: 168
counter: 169
counter: 170
counter: 171
counter: 172
counter: 173
counter: 174
counter: 175
counter: 176
counter: 177
counter: 178
counter: 179
counter: 180

4390
8178
3331
5280
counter: 1
counter: 2
6523
7757
counter: 2
counter: 3
counter: 4
counter: 5
counter: 6
counter: 7
counter: 8
counter: 9
counter: 10
counter: 11
counter: 12
counter: 13
counter: 14
counter: 15
counter: 16
counter: 17
counter: 18
counter: 19
counter: 20
counter: 21
counter: 22
counter: 23
counter: 24
counter: 25
counter: 26
counter: 27
counter: 28
counter: 29
counter: 30
counter: 31
counter: 32
counter: 33
counter: 34
counter: 35
counter: 36
counter: 37
counter: 38
counter: 39
counter: 40
counter: 41
counter: 42
counter: 43
counter: 44
counter: 45
counter: 46
counter: 47
counter: 48
counter: 49
counter: 50
counter: 51
counter: 52
counter: 53
counter: 54
counter: 55
counter: 56
counter: 57
counter: 58
counter: 59
counter: 60
counter: 61
counter: 62
counter: 63
counter: 64
counter: 65
counter: 66
counter: 67
counter: 68
counter: 69
counter: 70
counter: 71
counter: 72
counter: 73
counter: 74
counter: 75
counter: 76
counter: 77
counter: 78
counter: 79
counter: 80
counter:

counter: 157
counter: 158
counter: 159
counter: 160
counter: 161
counter: 162
counter: 163
counter: 164
counter: 165
counter: 166
counter: 167
counter: 168
counter: 169
counter: 170
counter: 171
counter: 172
counter: 173
counter: 174
counter: 175
counter: 176
counter: 177
counter: 178
counter: 179
counter: 180
counter: 181
counter: 182
counter: 183
counter: 184
counter: 185
counter: 186
counter: 187
counter: 188
counter: 189
counter: 190
counter: 191
counter: 192
counter: 193
counter: 194
counter: 195
counter: 196
counter: 197
counter: 198
counter: 199
counter: 200
counter: 201
counter: 202
counter: 203
counter: 204
counter: 205
counter: 206
counter: 207
counter: 208
counter: 209
counter: 210
counter: 211
counter: 212
counter: 213
counter: 214
counter: 215
counter: 216
counter: 217
counter: 218
counter: 219
counter: 220
counter: 221
counter: 222
counter: 223
counter: 224
counter: 225
counter: 226
counter: 227
counter: 228
counter: 229
counter: 230
counter: 231
counter: 232
counter: 233

KeyboardInterrupt: 

In [113]:
len(df_food_titles_similar)

167

In [75]:
df_food_titles_similar.iloc[-1]

food_post_title                                                    baby duck
closest_match_from_usda    [Baby Ruth, Duck, cooked, skin eatenDuck, cook...
calories_100g                                                         225.69
protein_100g                                                           17.15
fat_100g                                                                 3.4
sugar_100g                                                              5.49
carbohydrates_100g                                                      9.08
sodium_100g                                                           189.59
cholesterol_100g                                                      162.55
fiber_100g                                                              0.16
index_of_usda_in_json      [6715, 1280, 1281, 1282, 1283, 1284, 2856, 662...
Name: 0, dtype: object

In [93]:
df_food_titles_similar.to_csv('test_compare_v2.csv')

In [80]:
!pip install calorieninjas

ERROR: Could not find a version that satisfies the requirement calorieninjas (from versions: none)
ERROR: No matching distribution found for calorieninjas


In [83]:
from calorieninjas import query_for_foods, query_for_food, aggregate_nutrition_component_food_dict, open_connection, nutrition_dict_to_df

%load_ext autoreload
%autoreload 2

In [84]:
df_sampled_recipes = df_food_posts[:401]
df_sampled_recipes.head()

,Unnamed: 0,id,url,datetime,title,author,num_comments,thumbnail,domain,preprocessed_title,preprocessed_title_nouns,preprocessed_title_nouns_no_duplicates,preprocessed_title_nouns_final
0,0,810ib4,https://i.redd.it/ynh08qx9b1j01.jpg,2018-03-01 00:01:28,[I ate] Thinly-sliced Japanese Wagyu,Mike11888,125,https://b.thumbs.redditmedia.com/lEroJcEue-KPY...,i.redd.it,ate thinlysliced japanese wagyu,wagyu,['wagyu'],[wagyu]
1,1,810ljv,https://i.redd.it/kobpdjsed1j01.jpg,2018-03-01 00:13:38,"I smoked a brisket and we made doughnuts, so t...",Chef6288,0,default,i.redd.it,smoked brisket doughnuts happened prochef,brisket doughnuts,"['brisket', 'doughnuts']","[brisket, doughnuts]"
2,2,810lvl,https://i.redd.it/ddugaidod1j01.jpg,2018-03-01 00:14:56,[I Ate] Matcha Mochi Melonpan,danceyourdeath,0,https://b.thumbs.redditmedia.com/EA7TutwVLakIt...,i.redd.it,ate matcha mochi melonpan,matcha mochi melonpan,"['matcha', 'mochi', 'melonpan']","[matcha, mochi, melonpan]"
3,3,810lx7,https://i.redd.it/yqax4gypd1j01.jpg,2018-03-01 00:15:10,[I made] this Wakanda/Black Panther inspired U...,[deleted],0,default,i.redd.it,wakandablack panther inspired unicorn cake,wakandablack panther cake,"['wakandablack', 'panther', 'cake']","[wakandablack, panther, cake]"
4,4,810m58,https://imgur.com/m9BDe1F,2018-03-01 00:16:01,Homemade Flax seed dough. Caper [homemade],PassionateSizzle,1,default,imgur.com,homemade flax seed dough caper homemade,flax seed caper,"['flax', 'seed', 'caper']","[flax, seed, caper]"


In [85]:
conn = open_connection()
results_sample = query_for_foods(conn, df_sampled_recipes.title)

In [86]:
nutrition_sample = aggregate_nutrition_component_food_dict(results_sample)
df_nutrition = nutrition_dict_to_df(nutrition_sample)
df_nutrition.head(10)

,food_title,calories,carbohydrates_total_g,cholesterol_mg,fat_saturated_g,fat_total_g,fiber_g,potassium_mg,protein_g,sodium_mg,sugar_g
0,Thinly-sliced Japanese Wagyu,190.7,0.000000,58.000000,5.500000,12.300000,0.000000,0.000000,20.000000,63.000000,0.000000
1,"I smoked a brisket and we made doughnuts, so t...",351.0,24.250000,68.500000,8.400000,20.550000,1.050000,144.000000,17.650000,183.500000,11.450000
2,Matcha Mochi Melonpan,132.5,26.450000,2.000000,2.250000,2.800000,0.250000,36.500000,1.350000,9.500000,18.000000
3,this Wakanda/Black Panther inspired Unicorn Cake,393.6,57.200000,74.000000,2.900000,17.600000,0.300000,142.000000,3.000000,271.000000,42.100000
4,Homemade Flax seed dough. Caper,330.2,28.333333,0.000000,4.533333,23.133333,10.700000,234.666667,7.833333,918.666667,0.666667
5,طريقة عمل عصير الكرز المركز لذيذ ومغذي,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Melon favoured shaved ice with ice cream,85.8,12.066667,14.666667,2.300000,3.733333,0.533333,39.333333,1.433333,32.000000,10.700000
7,Shake Shack. Double Shack Burger. Look at this...,178.8,19.800000,32.000000,3.200000,7.100000,0.150000,132.000000,9.150000,233.500000,10.450000
8,BBQ Chicken Mac n Cheese,181.7,17.850000,27.500000,1.550000,6.450000,0.750000,124.500000,11.400000,341.500000,6.950000
9,Fried Deviled Eggs,222.2,1.600000,312.000000,4.400000,19.000000,0.400000,168.000000,10.600000,228.000000,0.400000


In [87]:
df_sampled_recipes = pd.concat([df_sampled_recipes, df_nutrition], axis=1)
df_sampled_recipes.head(10)

,Unnamed: 0,id,url,datetime,title,author,num_comments,thumbnail,domain,preprocessed_title,...,calories,carbohydrates_total_g,cholesterol_mg,fat_saturated_g,fat_total_g,fiber_g,potassium_mg,protein_g,sodium_mg,sugar_g
0,0.0,810ib4,https://i.redd.it/ynh08qx9b1j01.jpg,2018-03-01 00:01:28,[I ate] Thinly-sliced Japanese Wagyu,Mike11888,125.0,https://b.thumbs.redditmedia.com/lEroJcEue-KPY...,i.redd.it,ate thinlysliced japanese wagyu,...,190.7,0.000000,58.000000,5.500000,12.300000,0.000000,0.000000,20.000000,63.000000,0.000000
1,1.0,810ljv,https://i.redd.it/kobpdjsed1j01.jpg,2018-03-01 00:13:38,"I smoked a brisket and we made doughnuts, so t...",Chef6288,0.0,default,i.redd.it,smoked brisket doughnuts happened prochef,...,351.0,24.250000,68.500000,8.400000,20.550000,1.050000,144.000000,17.650000,183.500000,11.450000
2,2.0,810lvl,https://i.redd.it/ddugaidod1j01.jpg,2018-03-01 00:14:56,[I Ate] Matcha Mochi Melonpan,danceyourdeath,0.0,https://b.thumbs.redditmedia.com/EA7TutwVLakIt...,i.redd.it,ate matcha mochi melonpan,...,132.5,26.450000,2.000000,2.250000,2.800000,0.250000,36.500000,1.350000,9.500000,18.000000
3,3.0,810lx7,https://i.redd.it/yqax4gypd1j01.jpg,2018-03-01 00:15:10,[I made] this Wakanda/Black Panther inspired U...,[deleted],0.0,default,i.redd.it,wakandablack panther inspired unicorn cake,...,393.6,57.200000,74.000000,2.900000,17.600000,0.300000,142.000000,3.000000,271.000000,42.100000
4,4.0,810m58,https://imgur.com/m9BDe1F,2018-03-01 00:16:01,Homemade Flax seed dough. Caper [homemade],PassionateSizzle,1.0,default,imgur.com,homemade flax seed dough caper homemade,...,330.2,28.333333,0.000000,4.533333,23.133333,10.700000,234.666667,7.833333,918.666667,0.666667
5,5.0,810nxf,https://elbanota.com/%d8%b7%d8%b1%d9%8a%d9%82%...,2018-03-01 00:23:08,طريقة عمل عصير الكرز المركز لذيذ ومغذي,elbanota,0.0,default,elbanota.com,طريقة عمل عصير الكرز المركز لذيذ ومغذي,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,810ulz,https://i.redd.it/ihksb91zj1j01.jpg,2018-03-01 00:50:12,[I ate] Melon favoured shaved ice with ice cream,ShippinitlikeFedEx,2.0,https://b.thumbs.redditmedia.com/MUg8Re0F8kNFv...,i.redd.it,ate melon favoured shaved ice ice cream,...,85.8,12.066667,14.666667,2.300000,3.733333,0.533333,39.333333,1.433333,32.000000,10.700000
7,7.0,810w1p,https://i.redd.it/m0qlhwc1l1j01.jpg,2018-03-01 00:56:14,[I Ate] Shake Shack. Double Shack Burger. Look...,Be_Mighty,2.0,default,i.redd.it,ate shake shack double shack burger look maste...,...,178.8,19.800000,32.000000,3.200000,7.100000,0.150000,132.000000,9.150000,233.500000,10.450000
8,8.0,810w3z,https://i.imgur.com/cYZOBr5.jpg,2018-03-01 00:56:26,[Homemade] BBQ Chicken Mac n Cheese,abattleofone,20.0,https://b.thumbs.redditmedia.com/rxFiUtBAP97pN...,i.imgur.com,homemade bbq chicken mac n cheese,...,181.7,17.850000,27.500000,1.550000,6.450000,0.750000,124.500000,11.400000,341.500000,6.950000
9,9.0,810xrw,https://i.redd.it/u9zr4k7cm1j01.jpg,2018-03-01 01:03:29,[Pro/Chef] Fried Deviled Eggs,drawthings,25.0,https://b.thumbs.redditmedia.com/83tNsaLXjRgvV...,i.redd.it,prochef fried deviled eggs,...,222.2,1.600000,312.000000,4.400000,19.000000,0.400000,168.000000,10.600000,228.000000,0.400000


In [88]:
df_nutrition.to_csv('df_nutrition.csv')

In [89]:
df_sampled_recipes.to_csv('df_sampled_recipes.csv')